In [1]:
#cell0:
from pathlib import Path

DATA_DIR = Path("../data")
OUT_DIR = Path("../outputs")
LOG_DIR = Path("../logs")

# make sure folders exist
OUT_DIR.mkdir(exist_ok=True, parents=True)
LOG_DIR.mkdir(exist_ok=True, parents=True)

print("Folders ready!")


Folders ready!


In [2]:

# Cell 1 
import sys
import os

# add project root to Python path
sys.path.append(os.path.abspath(".."))

import logging
from pathlib import Path
import pandas as pd

# set up logging
logging.basicConfig(filename='logs/energy.log', level=logging.INFO,
                    format='%(asctime)s - %(levelname)s - %(message)s')

DATA_DIR = Path('../data')  # if notebook inside notebooks/
OUT_DIR = Path('../outputs')
OUT_DIR.mkdir(parents=True, exist_ok=True)

from src.ingest import read_all
from src.aggregates import calculate_daily_totals, calculate_weekly_aggregates, building_wise_summary
from src.models import BuildingManager

# Import the viz module to check available functions
import src.viz
# Print available functions in the viz module to find the correct function name
print(dir(src.viz))

# Then import the correct function based on what's available
# For example, if there's a function called 'plot_dashboard' instead:
# from src.viz import plot_dashboard as dashboard_plot

['__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__', 'plot_usage', 'plt']


In [3]:
#cell2
import sys
sys.path.append("..")

from src.ingest import read_all
from src.aggregates import (
    calculate_daily_totals,
    calculate_weekly_aggregates,
    building_wise_summary
)
from src.models import BuildingManager
from src.viz import plot_usage

print("Modules imported successfully!")


Modules imported successfully!


In [4]:
#cell3
df, errors = read_all(DATA_DIR)

print("Loaded DataFrame:")
display(df.head())

print("\nAny file errors:")
print(errors)


Loaded DataFrame:


,timestamp,kwh,building
0,2024-01-01 00:00:00,20,BuildingB_2024
1,2024-01-01 01:00:00,22,BuildingB_2024
2,2024-01-01 02:00:00,19,BuildingB_2024
3,2024-01-01 03:00:00,21,BuildingB_2024
4,2024-01-01 04:00:00,18,BuildingB_2024



Any file errors:
[]


In [5]:
#cell4
daily = calculate_daily_totals(df)
print("Daily Totals:")
display(daily.head())


Daily Totals:


,building,timestamp,kwh
0,BuildingA_2024,2024-01-01,55
1,BuildingA_2024,2024-01-02,51
2,BuildingB_2024,2024-01-01,100
3,BuildingB_2024,2024-01-02,89
4,BuildingC_2024,2024-01-01,42


In [6]:
#cell5
weekly = calculate_weekly_aggregates(df)
print("Weekly Aggregates:")
display(weekly.head())


Weekly Aggregates:


,building,timestamp,kwh
0,BuildingA_2024,2024-01-07,106
1,BuildingB_2024,2024-01-07,189
2,BuildingC_2024,2024-01-07,84


In [7]:
#cell6
import pandas as pd

# Create or load your DataFrame
# Option 1: Create a sample DataFrame
df = pd.DataFrame({
    'building': ['A', 'B', 'A', 'C', 'B', 'A'],
    'value': [10, 20, 15, 30, 25, 5]
})

# Option 2: Load from a file (uncomment and modify as needed)
# df = pd.read_csv('your_file.csv')

# Now use the DataFrame in your function
def building_wise_summary(dataframe):
    # Assuming this function groups by 'building' column
    return dataframe.groupby('building').sum()

summary = building_wise_summary(df)
print("Building-wise Summary:")
display(summary)

Building-wise Summary:


,value
building,
A,30
B,45
C,30


In [8]:

import pandas as pd

# Create or load your DataFrame
# Option 1: Create a sample DataFrame
df = pd.DataFrame({
    'building': ['A', 'B', 'A', 'C', 'B', 'A'],
    'value': [10, 20, 15, 30, 25, 5]
})

# Option 2: Load from a file (uncomment and modify as needed)
# df = pd.read_csv('your_file.csv')

# Now use the DataFrame in your function
def building_wise_summary(dataframe):
    # Assuming this function groups by 'building' column
    return dataframe.groupby('building').sum()

summary = building_wise_summary(df)
print("Building-wise Summary:")
display(summary)

Building-wise Summary:


,value
building,
A,30
B,45
C,30


In [9]:
df.columns


Index(['building', 'value'], dtype='object')

In [10]:
df = df.rename(columns={
    'value': 'kwh'
})

# Add a timestamp column since your data does NOT have one
import pandas as pd

df['timestamp'] = pd.date_range(start='2024-01-01', periods=len(df), freq='H')

df = df[['timestamp', 'kwh', 'building']]   # reorder columns
df.head()


/var/folders/yb/5z1wj0qs34g0js54d25_d1g40000gn/T/ipykernel_3427/3631690835.py:8: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df['timestamp'] = pd.date_range(start='2024-01-01', periods=len(df), freq='H')


,timestamp,kwh,building
0,2024-01-01 00:00:00,10,A
1,2024-01-01 01:00:00,20,B
2,2024-01-01 02:00:00,15,A
3,2024-01-01 03:00:00,30,C
4,2024-01-01 04:00:00,25,B


In [11]:
#cell7
bm = BuildingManager(df, energy_column="kwh", building_column="building")

print("Total energy:")
print(bm.total_energy())

print("\nTop building by usage:")
print(bm.top_building())

print("\nLowest building:")
print(bm.lowest_building())


Total energy:
105

Top building by usage:
('B', 45)

Lowest building:
('A', 30)


In [12]:
#cell8
# Plot energy usage trend
from src.viz import plot_usage

print("Generating usage plot...")
plot_usage(df, save_path="../outputs/energy_usage.png")


Generating usage plot...
Plotting error: 'Date'


In [14]:
#cell9
daily.to_csv(OUT_DIR / "daily_usage.csv", index=False)
weekly.to_csv(OUT_DIR / "weekly_usage.csv", index=False)
summary.to_csv(OUT_DIR / "building_summary.csv", index=True)

print("All outputs saved to /outputs folder.")


All outputs saved to /outputs folder.
